# 학습에 도움이 되는 테크닉

## 1. Learning Rate Scheduler

학습 중 학습률(learning rate, LR)을 조정

 ![image.png](http://drive.google.com/uc?id=1LsHW_g90NRBemBEm-1Bn-0et0qDNyi8v)

 ![image.png](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbeNGrz%2FbtqQzRMQAQx%2FE5HgPj8oe4AugHgHxS7RlK%2Fimg.png)

### optim.lr_scheduler
 - [doc] (https://pytorch.org/docs/stable/optim.html)

 ![image.png](http://drive.google.com/uc?id=1Ri1piSSdPng993SCk0vGqDGuoiD5aDqd)

 ![image.png](http://drive.google.com/uc?id=1cU3Et1QbKPf4jKoCwK14jDz3SgVkRG8N)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch.optim import SGD, lr_scheduler

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)


    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [ ]:
lr = 0.01
step_size = 30
gamma = 0.1

model = Net()

# 옵티마이저 및 스케쥴러 설정
optimizer = SGD(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma) # step_size마다 학습률에 gamma를 곱함

lrs = []
for i in range(100):
    lrs.append(scheduler.get_last_lr()[0])
    optimizer.step()
    scheduler.step() # 스케쥴러를 사용하기 위해서 필수적으로 들어가야함

plt.plot(range(100), lrs)
plt.show()

## 2. Early Stopping

Validation 데이터셋을 두고, validation 데이터셋의 예측 결과를 통해 학습 중단 시기를 결정하는 기법

![image.png](https://miro.medium.com/max/1400/1*06sTlOC3AYeZAjzUDwbaMw@2x.jpeg)

In [ ]:
import torch

In [ ]:
# [주의] 아래는 알고리즘 참고용이며 이대로는 실행되지 않음

count = 0
best_acc = 0
patience = 3

for _ in range(100000):
    # 1회 학습
    model.train()
    for input, target in train_loader:
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    model.eval()
    valid_acc = 0
    with torch.no_grad():
        for input, target in valid_loader:
            output = model(input)
            valid_acc += accuracy(output, target)
        valid_acc /= len(valid_loader) # 지정한 퍼포먼스 메트릭으로 성능 평가 (편의상 accuracy로 표현)

    if valid_acc > best_acc: # 더 좋은 성능이 나온다면
        best_acc = valid_acc # 최고 성능 업데이트
        count = 0 # 카운트 초기화
        torch.save(model.state_dict(), './model.pt') # 모델 저장
    else: # 성능이 오르지 않는다면
        count += 1 # 카운트 추가
        if count >= patience: # 참기로 한 횟수(patience)만큼 성능이 오르지 않았다면
            break # 학습을 멈춤

## 3. Dropout : `nn.Dropout2d`
학습 중 뉴런을 일시적으로 제거(drop)하는 기법
- Dropout rate가 0.5인 경우, 4개의 뉴런은 각각 0.5의 확률로 이번 학습에서 제거될지 말지를 결정함
  - [내용 참고](https://heytech.tistory.com/127)

![imgae.png](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fm5YtD%2FbtrbwgszFXZ%2FxdUhkeyK2lmrQXYy7HioL0%2Fimg.png)

- [doc] (https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html)

 ![image.png](http://drive.google.com/uc?id=1qYq4XLqNskL8ejGVjFblS9cVfYIJmyHR)
 ![image.png](http://drive.google.com/uc?id=11jcEX8ADdeW4y2U1EnFbk0j2h4OQX9TG)

아래 지시에 따라 코드를 완성해주세요

- [N, C, H, W] = [N, 1, 32, 32] 사이즈의 입력이 들어갈 예정
- 예측 클래스는 8개
- 3개의 fully connected layer로 구성됨 (fc1, fc2, fc3)
    - 활성함수는 ReLU를 사용하며, 마지막 레이어에는 활성함수가 없음
- 첫 dropout은 input layer에 적용함
    - Dropout은 25% 확률로 실행함
- 두번째 dropout은 fc2에 적용함
    - Dropout은 20% 확률로 실행함

In [ ]:
# 필요한 import를 작성해주세요


In [ ]:
# 모델을 구현해주세요


위와 같은 모델을:
단, `nn.Dropout`이 아닌 `torch.nn.functional.dropout`을 써서 구현해주세요

## 4. Batch Normalization

- 학습 과정에서 각 배치 단위 별로 데이터가 다양한 분포를 가지더라도 각 배치별로 평균과 분산을 이용해 정규화하는 것을 뜻함
- batch 단위나 layer에 따라서 입력 값의 분포가 모두 다르지만 정규화를 통하여 분포를 zero mean gaussian 형태로 만듦
    - [내용 참고](https://gaussian37.github.io/dl-concept-batchnorm/)

![image.png](https://gaussian37.github.io/assets/img/dl/concept/batchnorm/4.png)

- [doc] (https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)

 ![image.png](http://drive.google.com/uc?id=1sId2CSNqxKncTueKwgtIoBzEMvSfAvhx)
 ![image.png](http://drive.google.com/uc?id=1fycDf_67C9OBV1GHZBmwD4P83iNtS5XL)

아래 코드는 에러가 나니 잘못된 부분을 찾아 수정해 보세요

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32 * 32, 256)
        self.bn1 = nn.BatchNorm1d(32 * 32)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm2d(128)
        self.fc3 = nn.Linear(64, 4)

    def forward(self, x):
        # 이 부분은 문제 없음
        x = x.view(-1, 32 * 32)
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        return self.fc3(x)

In [ ]:
# 버그를 잘 잡았는지 아래 코드를 실행하여 확인
## 문제가 없다면 에러 없이 실행됨

model = Net()
model(torch.rand(2, 32, 32))